In [ ]:
import numpy as np #linear algebra
import pandas as pd #data processing 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)


In [ ]:
df_IMDB_Title_Akas = pd.read_csv("./Prj_Data/DownLoadedData_Imdb/imdb_title_akas.tsv", delimiter='\t')
df_IMDB_Title_Akas.name = "df_IMDB_Title_Akas"

df_IMDB_Title_Akas = df_IMDB_Title_Akas.loc[(df_IMDB_Title_Akas['titleType']=='movie')]
df_IMDB_Title_Akas = df_IMDB_Title_Akas.loc[(df_IMDB_Title_Akas['isAdult']==0)]
df_IMDB_Title_Akas = df_IMDB_Title_Akas.loc[~(df_IMDB_Title_Akas['genres']=='\\N')]
df_IMDB_Title_Akas.drop(['endYear', 'isAdult'], axis=1, inplace = True)

df_IMDB_Title_Akas["startYear"] = df_IMDB_Title_Akas.startYear.replace(r'\N',"0", regex=False)
df_IMDB_Title_Akas['startYear'] = df_IMDB_Title_Akas['startYear'].astype('int32')


df_IMDB_Title_Akas["runtimeMinutes"] = df_IMDB_Title_Akas.runtimeMinutes.replace(r'\N',"0", regex=False)
df_IMDB_Title_Akas['runtimeMinutes'] = df_IMDB_Title_Akas['runtimeMinutes'].astype('float64')

# df_IMDB_Title_Akas = df_IMDB_Title_Akas[df_IMDB_Title_Akas.primaryTitle.str.isalpha()]
df_IMDB_Title_Akas.dropna(subset=['primaryTitle'], how='all', inplace=True)
df_IMDB_Title_Akas.fillna({"startYear":0,"runtimeMinutes":0}, inplace=True)
df_IMDB_Title_Akas = df_IMDB_Title_Akas.loc[(df_IMDB_Title_Akas['startYear']>=2005) & (df_IMDB_Title_Akas['startYear']<=2020)]


df_IMDB_Title_Akas = df_IMDB_Title_Akas[df_IMDB_Title_Akas['primaryTitle'].str.contains('[A-Za-z0-9:-]', na=False, regex=True)]
df_IMDB_Title_Akas = df_IMDB_Title_Akas.loc[~(df_IMDB_Title_Akas['primaryTitle'].str.contains('á|à|â|ã|é|è|ê|í|ï|ó|ô|õ|ö|ú|ç|ñ', case=False))]


df_IMDB_Title_Ratings = pd.read_csv("./Prj_Data/DownLoadedData_Imdb/Ratings.tsv", delimiter='\t')
df_Imdb_MoviesWithRatings = df_IMDB_Title_Akas.merge(df_IMDB_Title_Ratings, on='tconst', how='inner', suffixes=('_AK', '_rat'))


In [ ]:
df_Imdb_MoviesWithRatings["Splitgeneres"] = df_Imdb_MoviesWithRatings["genres"].str.split(",",3)
df_Imdb_MoviesWithRatings_Exploded = df_Imdb_MoviesWithRatings.explode("Splitgeneres")

In [ ]:
df_genres = df_Imdb_MoviesWithRatings_Exploded.pivot_table(index=['Splitgeneres'], values="averageRating", aggfunc={"averageRating":['count','mean','min','max','std']},dropna=True).reset_index()

df_genres = df_genres.sort_values(by="count", ascending=False)

df_genres = df_genres.loc[(df_genres['count']>23)]
df_genres

In [ ]:
#df_Imdb_MoviesWithRatings_Exploded
df_Imdb_MoviesWithRatings_Exploded = df_Imdb_MoviesWithRatings_Exploded.merge(df_genres, on='Splitgeneres', how='left', suffixes=('_exr', '_desc'))
df_Imdb_MoviesWithRatings_Exploded.head(100)

In [ ]:
df_Imdb_MoviesWithRatings_Exploded.info()
df_Imdb_MoviesWithRatings_Exploded_Above = df_Imdb_MoviesWithRatings_Exploded.loc[(df_Imdb_MoviesWithRatings_Exploded['AboveRating']==True)]
df_Imdb_MoviesWithRatings_Exploded_Above.info()